In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer

2023-05-23 21:16:33.995528: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
school_final = pd.read_csv('school_final.csv')
school_final = school_final.drop('Unnamed: 0', axis=1)
school_final.head()

,Title,Author,publisher,description,published_date,categories,Type of Ban,State,District,Date of Challenge/Removal,Origin of Challenge,State_Count
0,Ace of Spades,"Àbíké-Íyímídé, Faridah",Feiwel & Friends,"Gossip Girl meets Get Out in Ace of Spades, a ...",2021-06-01,['Young Adult Fiction'],Banned in Libraries and Classrooms,Florida,Indian River County School District,November 2021,Administrator,923
1,Clap When You Land,"Acevedo, Elizabeth",HarperCollins,In a novel-in-verse that brims with grief and ...,2020-05-05,['Young Adult Fiction'],Banned in Classrooms,Pennsylvania,Central York School District,August 2021,Administrator,485
2,The Poet X,"Acevedo, Elizabeth",HarperCollins,A National Book Award Longlist title! Fans of ...,2019-03-19,['Young Adult Fiction'],Banned in Libraries,Florida,Indian River County School District,November 2021,Administrator,923
3,The Poet X,"Acevedo, Elizabeth",HarperCollins,A National Book Award Longlist title! Fans of ...,2019-03-19,['Young Adult Fiction'],Banned in Libraries and Classrooms,New York,Marlboro Central School District,February 2022,Administrator,15
4,The Poet X,"Acevedo, Elizabeth",HarperCollins,A National Book Award Longlist title! Fans of ...,2019-03-19,['Young Adult Fiction'],Banned Pending Investigation,Texas,Fredericksburg Independent School District,March 2022,Administrator,1239


In [4]:
state_abbreviations = {
    'Florida': 'FL',
    'Pennsylvania': 'PA',
    'New York': 'NY',
    'Texas': 'TX',
    'Virginia': 'VA',
    'Michigan': 'MI',
    'Tennessee': 'TN',
    'Mississippi': 'MS',
    'Wisconsin': 'WI',
    'Idaho': 'ID',
    'Georgia': 'GA',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Oklahoma': 'OK',
    'Missouri': 'MO',
    'Ohio': 'OH',
    'Oregon': 'OR',
    'Rhode Island': 'RI',
    'South Dakota': 'SD',
    'Indiana': 'IN',
    'Vermont': 'VT',
    'North Carolina': 'NC',
    'Minnesota': 'MN',
    'Utah': 'UT',
    'Maryland': 'MD',
    'Washington': 'WA',
    'Alaska': 'AK',
    'Illinois': 'IL',
    'Maine': 'ME',
    'South Carolina': 'SC',
    'Arkansas': 'AR',
    'New Jersey': 'NJ',
    'California': 'CA',
    'Colorado': 'CO',
    'Massachusetts': 'MA',
    'North Dakota': 'ND',
    'Wyoming': 'WY'
}

In [5]:
# separate each state into its own dataframe
for state in state_abbreviations:
    abbreviation = state_abbreviations[state]
    locals()[abbreviation] = school_final[school_final['State'] == state]

In [6]:
# create copies 
FL_copy = FL.copy()
PA_copy = PA.copy()
NY_copy = NY.copy()
TX_copy = TX.copy()
VA_copy = VA.copy()
MI_copy = MI.copy()
TN_copy = TN.copy()
MS_copy = MS.copy()
WI_copy = WI.copy()
ID_copy = ID.copy()
GA_copy = GA.copy()
IA_copy = IA.copy()
KS_copy = KS.copy()
OK_copy = OK.copy()
MO_copy = MO.copy()
OH_copy = OH.copy()
OR_copy = OR.copy()
RI_copy = RI.copy()
SD_copy = SD.copy()
IN_copy = IN.copy()
VT_copy = VT.copy()
NC_copy = NC.copy()
MN_copy = MN.copy()
UT_copy = UT.copy()
MD_copy = MD.copy()
WA_copy = WA.copy()
AK_copy = AK.copy()
IL_copy = IL.copy()
ME_copy = ME.copy()
SC_copy = SC.copy()
AR_copy = AR.copy()
NJ_copy = NJ.copy()
CA_copy = CA.copy()
CO_copy = CO.copy()
MA_copy = MA.copy()
ND_copy = ND.copy()
WY_copy = WY.copy()

In [7]:
FL_copy['description'] = FL_copy['description'].str.lower()

words_to_remove = ["she's","he's",'book', 'story', 'world', 'best', 'new', 'york', 'times', 'author', 'novel', 'bestselling']


FL_copy['description'] = FL_copy['description'].apply(
    lambda text: ' '.join([word for word in text.split() if word not in words_to_remove])
)

In [10]:
docs = FL_copy.description.values

In [11]:
from bertopic.representation import KeyBERTInspired
from bertopic.representation import MaximalMarginalRelevance

vectorizer_model = CountVectorizer(ngram_range=(1, 1), stop_words='english')

embedding_model = SentenceTransformer("all-MiniLM-L6-v2") #all-MiniLM-L6-v2 #all-mpnet-base-v2
#representation_model = MaximalMarginalRelevance(diversity=0.5)
#representation_model = KeyBERTInspired()
mmr = MaximalMarginalRelevance(diversity=0.5)
representation_model = [mmr, KeyBERTInspired()]

topic_model_fl = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    representation_model=representation_model,
    language='english', 
    calculate_probabilities=True,
    verbose=True
)
topics, probs = topic_model_fl.fit_transform(docs)

topic_model_fl.get_topic_info()

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

2023-05-23 21:25:03,973 - BERTopic - Transformed documents to Embeddings
2023-05-23 21:25:10,941 - BERTopic - Reduced dimensionality
2023-05-23 21:25:11,016 - BERTopic - Clustered reduced embeddings


,Topic,Count,Name
0,-1,335,-1_novels_bestseller_fiction_vonnegut
1,0,80,0_molly_cassie_nina_leah
2,1,71,1_queer_johnson_slavery_racial
3,2,61,2_lgbt_transgender_lgbtq_sexuality
4,3,36,3_seductive_dazzling_fae_sarah
5,4,35,4_deogratias_poetry_poems_dreams
6,5,29,5_novel_pecola_poetry_beauty
7,6,27,6_jazz_transgender_teens_teen
8,7,25,7_vampire_avenge_bloodlines_hunt
9,8,20,8_sundance_greg_bestseller_andrews


In [12]:
# assign highest probability to outliers and group them with most similar topic
probability_threshold = 0.01
new_topics = [np.argmax(prob) if max(prob) >= probability_threshold else -1 for prob in probs]

topic_model_fl.update_topics(docs, topics=new_topics, vectorizer_model=vectorizer_model)
topic_model_fl.get_topic_info()

,Topic,Count,Name
0,-1,68,-1_ebay_susan_la_del
1,0,143,0_love_school_year_girl
2,1,69,1_black_history_racism_racist
3,2,75,2_gender_sex_gay_body
4,3,44,3_court_feyre_maas_series
5,4,85,4_time_american_vonnegut_family
6,5,31,5_grayson_morrison_blue_acclaimed
7,6,27,6_jazz_transgender_parents_jennings
8,7,14,7_zoey_neferet_house_vampyre
9,8,22,8_andrews_greg_haters_funny


In [13]:
# Step 1: Get topic information from the topic model and exclude outliers
topic_df = topic_model_fl.get_topic_info()[1:]
#topic_df = topic_model_mo.get_topic_info()

# Step 2: Create a DataFrame with descriptions and corresponding topics
topics_df = pd.DataFrame({"description": docs, "Topic": topic_model_fl.topics_})

# Step 3: Merge topics DataFrame with topic information DataFrame
merged_df = topics_df.merge(topic_df, on='Topic', how='left')

# Step 4: Merge fl DataFrame with merged_df using concat
merged_fl = pd.concat([FL.reset_index(drop=True), merged_df['Name'].reset_index(drop=True)], axis=1)

# Step 5: Remove the number and underscore from the 'Name' column
merged_fl['Topic'] = merged_fl['Name'].str.replace(r'^\d+_', '', regex=True)

merged_fl

,Title,Author,publisher,description,published_date,categories,Type of Ban,State,District,Date of Challenge/Removal,Origin of Challenge,State_Count,Name,Topic
0,Ace of Spades,"Àbíké-Íyímídé, Faridah",Feiwel & Friends,"Gossip Girl meets Get Out in Ace of Spades, a ...",2021-06-01,['Young Adult Fiction'],Banned in Libraries and Classrooms,Florida,Indian River County School District,November 2021,Administrator,923,0_love_school_year_girl,love_school_year_girl
1,The Poet X,"Acevedo, Elizabeth",HarperCollins,A National Book Award Longlist title! Fans of ...,2019-03-19,['Young Adult Fiction'],Banned in Libraries,Florida,Indian River County School District,November 2021,Administrator,923,4_time_american_vonnegut_family,time_american_vonnegut_family
2,"How I Paid for College: A Novel of Sex, Theft,...","Acito, Marc",Bloomsbury Publishing,'A comic coming-of-age novel ... it's an exube...,2009-08-17,['Fiction'],Banned Pending Investigation,Florida,Indian River County School District,November 2021,Administrator,923,8_andrews_greg_haters_funny,andrews_greg_haters_funny
3,My Name Is María Isabel,"Ada, Alma Flor",NaN,"Third grader Marâia Isabel, born in Puerto Ric...",1995-09,['Juvenile Fiction'],Banned Pending Investigation,Florida,Duval County Public Schools,January 2022,Administrator,923,4_time_american_vonnegut_family,time_american_vonnegut_family
4,Time to Pray,"Addasi, Maha","Christian Faith Publishing, Inc.",A visit with Grandmother in the Middle East is...,2010-09-01,['Religion'],Banned Pending Investigation,Florida,Duval County Public Schools,January 2022,Administrator,923,4_time_american_vonnegut_family,time_american_vonnegut_family
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,A Court of Mist and Fury,"Maas, Sarah J.",Bloomsbury Publishing USA,The seductive and stunning #1 New York Times b...,2016-05-03,['Fiction'],Banned in Libraries and Classrooms,Florida,Volusia County Schools,August 2022,Formal Challenge,923,3_court_feyre_maas_series,court_feyre_maas_series
919,A ​Court of Silver Flames,"Maas, Sarah J.",Bloomsbury Publishing USA,"Sarah J. Maas's sexy, richly imagined series c...",2021-02-16,['Fiction'],Banned in Libraries and Classrooms,Florida,Volusia County Schools,August 2022,Formal Challenge,923,3_court_feyre_maas_series,court_feyre_maas_series
920,A Court of Thorns and Roses,"Maas, Sarah J.",Bloomsbury Publishing USA,"Sarah J. Maas's sexy, richly imagined series c...",2021-02-16,['Fiction'],Banned in Libraries and Classrooms,Florida,Volusia County Schools,August 2022,Formal Challenge,923,3_court_feyre_maas_series,court_feyre_maas_series
921,A Court of Wings and Ruin,"Maas, Sarah J.",Bloomsbury Publishing USA,The epic third novel in the #1 New York Times ...,2017-05-02,['Fiction'],Banned in Libraries and Classrooms,Florida,Volusia County Schools,August 2022,Formal Challenge,923,3_court_feyre_maas_series,court_feyre_maas_series


In [ ]:
# final concatenation when done
state_dfs = [FL, PA, NY, TX, VA, MI, TN, MS, WI, ID, GA, IA, KS, OK, MO, OH, OR, RI, SD, IN, VT, NC, MN, UT, MD, WA, AK, IL, ME, SC, AR, NJ, CA, CO, MA, ND, WY]
combined_df = pd.concat(state_dfs)
combined_df